In [34]:
# Initial imports
import os
import pandas as pd
from datetime import datetime, timedelta
from pathlib import Path
from newsapi.newsapi_client import NewsApiClient
from dotenv import load_dotenv
from nltk.corpus import stopwords, reuters
from nltk.tokenize import sent_tokenize, word_tokenize
load_dotenv()
import nltk as nltk
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
nltk.download('punkt')
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import re

%matplotlib inline

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Admin\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [35]:
api_key = os.getenv("news_api")

In [36]:
newsapi = NewsApiClient(api_key=api_key)

In [37]:
tech_news = newsapi.get_everything(q = "Technology & Regulation", language="en", 
page_size=100, sort_by="relevancy")
print(tech_news["articles"][0])


{'source': {'id': 'engadget', 'name': 'Engadget'}, 'author': 'Igor Bonifacic', 'title': 'Automakers must report crashes involving self-driving and driver-assist systems', 'description': 'The National Highway Traffic Safety Administration (NHTSA) has implemented a new policy that will require car companies to report incidents involving semi- and fully autonomous driving systems within one day of learning of an accident. In an\xa0order\xa0spotted by\xa0…', 'url': 'https://www.engadget.com/nhtsa-automated-driving-system-crash-report-174552853.html', 'urlToImage': 'https://s.yimg.com/os/creatr-images/2020-01/1e88d580-3edd-11ea-8a57-ce70dbfb73e6', 'publishedAt': '2021-06-29T17:45:52Z', 'content': 'The National Highway Traffic Safety Administration (NHTSA) has implemented a new policy that will require car companies to report incidents involving semi- and fully autonomous driving systems within… [+1815 chars]'}


In [38]:
print(tech_news["totalResults"])

3531


In [39]:
#print(f"Total articles about Inflation: {inflation_news['totalResults']}")
#tech_news["articles"][0]

In [40]:
article_new = tech_news["articles"][0]

In [41]:
print(article_new)

{'source': {'id': 'engadget', 'name': 'Engadget'}, 'author': 'Igor Bonifacic', 'title': 'Automakers must report crashes involving self-driving and driver-assist systems', 'description': 'The National Highway Traffic Safety Administration (NHTSA) has implemented a new policy that will require car companies to report incidents involving semi- and fully autonomous driving systems within one day of learning of an accident. In an\xa0order\xa0spotted by\xa0…', 'url': 'https://www.engadget.com/nhtsa-automated-driving-system-crash-report-174552853.html', 'urlToImage': 'https://s.yimg.com/os/creatr-images/2020-01/1e88d580-3edd-11ea-8a57-ce70dbfb73e6', 'publishedAt': '2021-06-29T17:45:52Z', 'content': 'The National Highway Traffic Safety Administration (NHTSA) has implemented a new policy that will require car companies to report incidents involving semi- and fully autonomous driving systems within… [+1815 chars]'}


In [42]:
# In order to tokenize properly, all text needs to be strings, not integers or floats
article_letters_only = re.sub("[^a-zA-Z]", " ", str(article_new))
article_letters_only

'  source     id    engadget    name    Engadget     author    Igor Bonifacic    title    Automakers must report crashes involving self driving and driver assist systems    description    The National Highway Traffic Safety Administration  NHTSA  has implemented a new policy that will require car companies to report incidents involving semi  and fully autonomous driving systems within one day of learning of an accident  In an xa order xa spotted by xa      url    https   www engadget com nhtsa automated driving system crash report           html    urlToImage    https   s yimg com os creatr images          e  d     edd   ea  a   ce  dbfb  e     publishedAt              T        Z    content    The National Highway Traffic Safety Administration  NHTSA  has implemented a new policy that will require car companies to report incidents involving semi  and fully autonomous driving systems within         chars   '

In [43]:
# First, we tokenize sentences
sent_sample = sent_tokenize(article_letters_only)
print(sent_sample)

['  source     id    engadget    name    Engadget     author    Igor Bonifacic    title    Automakers must report crashes involving self driving and driver assist systems    description    The National Highway Traffic Safety Administration  NHTSA  has implemented a new policy that will require car companies to report incidents involving semi  and fully autonomous driving systems within one day of learning of an accident  In an xa order xa spotted by xa      url    https   www engadget com nhtsa automated driving system crash report           html    urlToImage    https   s yimg com os creatr images          e  d     edd   ea  a   ce  dbfb  e     publishedAt              T        Z    content    The National Highway Traffic Safety Administration  NHTSA  has implemented a new policy that will require car companies to report incidents involving semi  and fully autonomous driving systems within         chars']


In [44]:
# Second, we tokenize words within the sentences
word_sample = word_tokenize(article_letters_only)
print(word_sample)

['source', 'id', 'engadget', 'name', 'Engadget', 'author', 'Igor', 'Bonifacic', 'title', 'Automakers', 'must', 'report', 'crashes', 'involving', 'self', 'driving', 'and', 'driver', 'assist', 'systems', 'description', 'The', 'National', 'Highway', 'Traffic', 'Safety', 'Administration', 'NHTSA', 'has', 'implemented', 'a', 'new', 'policy', 'that', 'will', 'require', 'car', 'companies', 'to', 'report', 'incidents', 'involving', 'semi', 'and', 'fully', 'autonomous', 'driving', 'systems', 'within', 'one', 'day', 'of', 'learning', 'of', 'an', 'accident', 'In', 'an', 'xa', 'order', 'xa', 'spotted', 'by', 'xa', 'url', 'https', 'www', 'engadget', 'com', 'nhtsa', 'automated', 'driving', 'system', 'crash', 'report', 'html', 'urlToImage', 'https', 's', 'yimg', 'com', 'os', 'creatr', 'images', 'e', 'd', 'edd', 'ea', 'a', 'ce', 'dbfb', 'e', 'publishedAt', 'T', 'Z', 'content', 'The', 'National', 'Highway', 'Traffic', 'Safety', 'Administration', 'NHTSA', 'has', 'implemented', 'a', 'new', 'policy', 'tha

In [45]:
# Third, we get rid of stopwords that are not useful for proper tokenization. Here is a list.
print(stopwords.words('english'))


['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [46]:
# Fourth, we apply the stopwords function using the above stopwords list
sw_list = set(stopwords.words('english'))
sw_result = [word.lower() for word in word_sample if word.lower() not in sw_list]
print(sw_result)


['source', 'id', 'engadget', 'name', 'engadget', 'author', 'igor', 'bonifacic', 'title', 'automakers', 'must', 'report', 'crashes', 'involving', 'self', 'driving', 'driver', 'assist', 'systems', 'description', 'national', 'highway', 'traffic', 'safety', 'administration', 'nhtsa', 'implemented', 'new', 'policy', 'require', 'car', 'companies', 'report', 'incidents', 'involving', 'semi', 'fully', 'autonomous', 'driving', 'systems', 'within', 'one', 'day', 'learning', 'accident', 'xa', 'order', 'xa', 'spotted', 'xa', 'url', 'https', 'www', 'engadget', 'com', 'nhtsa', 'automated', 'driving', 'system', 'crash', 'report', 'html', 'urltoimage', 'https', 'yimg', 'com', 'os', 'creatr', 'images', 'e', 'edd', 'ea', 'ce', 'dbfb', 'e', 'publishedat', 'z', 'content', 'national', 'highway', 'traffic', 'safety', 'administration', 'nhtsa', 'implemented', 'new', 'policy', 'require', 'car', 'companies', 'report', 'incidents', 'involving', 'semi', 'fully', 'autonomous', 'driving', 'systems', 'within', 'cha

In [47]:
# Adding customized stopwords taken from the first result
sw_again = {'https', 'z', 'e', 'url'}
next_result = [word.lower() for word in word_sample if word.lower() not in sw.union(sw_again)]

In [48]:
print(next_result)

['source', 'id', 'engadget', 'name', 'engadget', 'author', 'igor', 'bonifacic', 'title', 'automakers', 'must', 'report', 'crashes', 'involving', 'self', 'driving', 'driver', 'assist', 'systems', 'description', 'national', 'highway', 'traffic', 'safety', 'administration', 'nhtsa', 'implemented', 'new', 'policy', 'require', 'car', 'companies', 'report', 'incidents', 'involving', 'semi', 'fully', 'autonomous', 'driving', 'systems', 'within', 'one', 'day', 'learning', 'accident', 'xa', 'order', 'xa', 'spotted', 'xa', 'www', 'engadget', 'com', 'nhtsa', 'automated', 'driving', 'system', 'crash', 'report', 'html', 'urltoimage', 'yimg', 'com', 'os', 'creatr', 'images', 'edd', 'ea', 'ce', 'dbfb', 'publishedat', 'content', 'national', 'highway', 'traffic', 'safety', 'administration', 'nhtsa', 'implemented', 'new', 'policy', 'require', 'car', 'companies', 'report', 'incidents', 'involving', 'semi', 'fully', 'autonomous', 'driving', 'systems', 'within', 'chars']


stopped here. Need to go back and push to github as proof that we did homework for 1st section of NLP